<a href="https://colab.research.google.com/github/Smartsie/Optimizer/blob/master/Optimizer_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
""" 
Optim package - T Martin 10 May 2020
This code if for multivariate optimizer using surrogate response surface created by several ML models
The next target point to study is either based on minimum point found, either by using a surrogate if r2 value of 
a model is above a certain threshold
"""

' \nOptim package - T Martin 10 May 2020\nThis code if for multivariate optimizer using surrogate response surface created by several ML models\nThe next target point to study is either based on minimum point found, either by using a surrogate if r2 value of \na model is above a certain threshold\n'

# **Functionnalities to be added**
1.   Models should be listed in a dictionnary. Each model is defined by it's function or class with train, predict and score 
2.   During fitting, we will score all models. The highest score will then be used for the prediction
3. This could be done by doing a voting of top 3 best models with a 3,2,1 weight for instance
4. Models accuracy will evolve with time (DOF)
5. Evaluation of models done on X-validation
6. Add early stop 







In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
import plotly as py
from keras.layers import Dense
import lightgbm
import plotly.express as px

Using TensorFlow backend.


In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
kernel = DotProduct() + WhiteKernel()
model = GaussianProcessRegressor(random_state=0)
import tensorflow.keras
from tensorflow.keras.models import Sequential
from xgboost import XGBRegressor

In [ ]:
# If we want to export in html in a separate browser
import plotly.io as pio
#pio.renderers.default = "browser" 

In [ ]:
def Japan_map():
  from google.colab import drive
  drive.mount('/content/drive')
  df=pd.read_csv('/content/drive/My Drive/Colab Data/Japan_XYZ_data.txt',header=None,delimiter=r"\s+")
  df.columns=['x0','x1','z']
  # Flip the data in z so that Mt Fuji is minimum
  df['z']=-df['z']
  # Resample the Dataframe to speed things up
  # N=10000 --> we keep only 10000 points
  N=500000
  step=int(len(df)/N)
  DF=df[::step]

  print('Theoretical Minimum:')
  print(DF.iloc[np.argmin(DF.z)])
  minx0=DF.iloc[np.argmin(DF.z)][0]
  minx1=DF.iloc[np.argmin(DF.z)][1]
  return

In [ ]:
def eval_f(vector):
  x1std=DF['x0'].std()
  x2std=DF['x1'].std()
  mask1=np.abs(DF['x0']-vector[0])<x1std/4
  mask2=np.abs(DF['x1']-vector[1])<x2std/4
  a=DF[mask1&mask2]
  result=a['z'].min()
  return result

In [ ]:
# Define function to minimize
def funct(x):
    x1=x[0]
    x2=x[1]
    #f=x1*x1+x1*x2+2*np.sin((x1+x2)*2)+4*np.cos(x2*2)
    f=x1*x1+x2*x2-x1*x2+20*np.sin(x1-x2)-8*np.cos(x2*2)
    f=(x1-3)*(x1-3)+(x2-1)*(x2-1)
    #f=3*x1*x1-2*x2+5*x1/x2
    ## Lévi function
    #f=(np.sin(3*3.1415*x1))**2+(x1-1)**2*(1+(np.sin(3*3.1415*x2))**2)+(x2-1)**2*(1+(np.sin(2*3.1415*x2))**2)
    # For Japan map f=eval_f(x)
    return f

In [ ]:
# Define the desX_ign space as center point with radius
x0range=[-5,5]
x1range=[-5,5]

In [ ]:

def Optimtm(plot_steps=True):
  # for japan x0range=[DF.x0.min(),DF.x0.max()]
  # x1range=[DF.x1.min(),DF.x1.max()]
  # This function adds the points to the matrix
  # The matrix is a dataframe
  def add_matrix(X,points):
      df=points
      for i in range(len(df)):
          variables=df.iloc[i]
          variables['z']=funct(variables)
          X=X.append(variables)
      return X
  # Define function providing N randoms points in the restraied domain 
  def next_point(current_space,n=5):
      next_space=current_space
      pt=pd.DataFrame()
      for i in range(n):
          vecteur=current_space['Center']
          Radius=current_space['Radius']
          new_vec=vecteur+Radius*np.random.choice((-1,1),size=len(Radius))*(np.random.random(len(Radius)))
          for j in range(len(new_vec)):
              if new_vec[j]<space.iloc[j]['Center']-space.iloc[j]['Radius']:
                  new_vec[j]=space.iloc[j]['Center']-space.iloc[j]['Radius']
              if new_vec[j]>space.iloc[j]['Center']+space.iloc[j]['Radius']:
                  new_vec[j]=space.iloc[j]['Center']+space.iloc[j]['Radius']                              
          pt['n'+str(i)]=new_vec
      pt=pt.T
      pt.columns=X.drop('z',axis=1).columns
      #next_space['Radius']=next_space['Radius']*epsilon
      return pt
  def training_loop(X,domain,AI=False,n=5,epsilon=0.95):
      df=next_point(domain,n)
      Matrix=add_matrix(X,df)
      x=Matrix.drop('z',axis=1).values
      y=Matrix['z'].values
      if AI==True:
          # Need to do a CV split with Train test split
          models={LinearRegression(), lightgbm.LGBMRegressor(),GaussianProcessRegressor(random_state=0)}
          # Additional models: MLPRegressor(hidden_layer_sizes=(5,3),max_iter=300, verbose=False,learning_rate_init=0.05),
          # XGBRegressor(objective ='reg:squarederror')
          X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.8,shuffle=True)
          r2max=-10
          best_model=LinearRegression()
          for model_loop in models:
            model_loop.fit(X_train,y_train)
            r2=model_loop.score(X_test,y_test)
            if r2>r2max:
              r2max=r2
              best_model=model_loop
          model=best_model
          # Retrain the model with all data
          model.fit(x,y)
          # Predict and find the minimum in one shot
          Prediction=model.predict(srf_x.values)
          lowest_value=np.min(Prediction)
          minimum=srf.iloc[np.argmin(Prediction)]    
          # Evaluate at minimum
          lowest_value=funct(minimum.values)    
          if r2max<0.7:
            #print('R2 insufficient %0.2f'%r2max)
            theor_min=np.argmin(X.z.values)
            lowest_value=np.min(X.z.values)
            minimum=X.iloc[theor_min]
          else:
              print("Model found !!!!!!!!! with r2=%0.2f"%r2max)
      else:
          theor_min=np.argmin(X.z.values)
          lowest_value=np.min(X.z.values)
          minimum=X.iloc[theor_min]
      minimum=minimum.drop('z')
      # New space should be centered around this minimum
      domain['Center']=(minimum.values*0.5+domain.Center*0.5)
      domain['Radius']=domain['Radius']*epsilon
      return Matrix,domain,lowest_value
  space=pd.DataFrame(columns=['Center','Radius'])
  space.loc[0]=[np.mean(x0range),0.5*(x0range[1]-x0range[0])]
  space.loc[1]=[np.mean(x1range),0.5*(x1range[1]-x1range[0])]
  # Initialize X matrix
  X=pd.DataFrame()
  for i in range(len(space)):
      X['x'+str(i)]=0
  X['z']=0
  # Fill X with 5 points
  X['x0']=[x0range[0],x0range[0],x0range[1],x0range[1],(x0range[0]+x0range[1])/2,(x0range[0]+x0range[1])/2,(x0range[0]+x0range[1])/2,x0range[0],x0range[1]]
  X['x1']=[x1range[0],x1range[1],x1range[0],x1range[1],(x1range[0]+x1range[1])/2,x1range[0],x1range[1],(x1range[0]+x1range[1])/2,(x1range[0]+x1range[1])/2]
  for i in range(len(X)):
      variables=X.iloc[i]
      #variables['target']=funct(variables[['x0','x1']].values)
      tgt=funct(variables.values)
      X.iloc[i]['z']=tgt

  # Initialize domain
  domain=space.copy()

  # Initialize Evaluation surface
  N=100
  srf=[]
  loc=0
  for i in np.linspace(x0range[0],x0range[1],N):
    for j in np.linspace(x1range[0],x1range[1],N):
      vector=[i,j,funct([i,j])]
      srf.append(vector)
  srf=pd.DataFrame(srf,columns=['x0','x1','z'])
  # for japan srf=DF
  srf_x=srf.drop(['z'],axis=1)


  plot_srf=False
  if plot_srf==True:
    fig = px.scatter_3d(srf, x='x0', y='x1', z='z',color='z')
    fig.show()
  print("Minimum : ",np.min(srf.z))
  print(srf.iloc[np.argmin(srf.z)])
  minx0=srf.iloc[np.argmin(srf.z)][0]
  minx1=srf.iloc[np.argmin(srf.z)][1]

  # Run the training loop
  N=20
  Max_points=20
  point_amount=Max_points
  Dom=[] #pd.DataFrame(columns=['x0 C','x0 R','x1 C','x1 R'])]
  for i in range(N):
      # Point amound should decrease. At the beginning, it should be 10. At the end, 2.
      #point_amount=np.int((Max_points-3)-(Max_points-3)*i/N)+3

      X,domain,minimum=training_loop(X,domain,AI=True,n=point_amount,epsilon=0.8)
      #print(i,minimum)
      #Dom=np.append(Dom,[i,domain.loc[0].values])
      vect=[i, domain.loc[0].values[0],domain.loc[0].values[1],domain.loc[1].values[0],domain.loc[1].values[1]]
      Dom.append(vect)
      if plot_steps==True:
        plt.scatter(X['x0'],X['x1'],color='blue')
        plt.scatter(Dom[-1][1],  Dom[-1][3],color='green')
        plt.scatter(minx0,minx1,color='red')
        plt.title("Loop "+str(i)+ "Minimum = %0.1f"%minimum)
        plt.show()
  Dom=pd.DataFrame(Dom,columns=['idx','x0 C','x0 R','x1 C','x1 R'])
  min_found=np.min(X.z.values)
  best_point=X.iloc[np.argmin(X.z.values)]
  return X,min_found, best_point

In [ ]:
X,min_found, best_points=Optimtm(False)

Minimum :  0.002040608101214206
x0    2.979798
x1    0.959596
z     0.002041
Name: 7959, dtype: float64


In [ ]:
# Print Minimum found
min=np.min(X.z.values)
print("Minimum found:")
print(X.iloc[np.argmin(X.z.values)])
#print("Theoretical Minimum : ")
#print(srf.iloc[np.argmin(srf.z)])


Minimum found:
x0    2.993185
x1    1.000014
z     0.000046
Name: n2, dtype: float64


In [ ]:
def plot_convergence():
  plt.plot(Dom.idx,Dom['x0 C'],color='red')
  plt.plot(Dom.idx,Dom['x0 C']+Dom['x0 R'],color='blue')
  plt.plot(Dom.idx,Dom['x0 C']-Dom['x0 R'],color='blue')
  plt.title('X0')
  plt.show()
  plt.plot(Dom.idx,Dom['x1 C'],color='red')
  plt.plot(Dom.idx,Dom['x1 C']+Dom['x1 R'],color='blue')
  plt.plot(Dom.idx,Dom['x1 C']-Dom['x1 R'],color='blue')
  plt.title('X1')
  plt.show()
  #plt.plot(Dom['x0 C'],Dom['x1 C'],color='red')
  plt.plot(Dom['x0 C'],Dom['x1 C'])
  plt.ylabel('X1')
  plt.xlabel('X0')
  plt.show()
  return

NameError: ignored

In [ ]:
# Using Hypertopt package
from hyperopt import hp, tpe, Trials, fmin,STATUS_OK

In [ ]:
space = {
    'x0': hp.uniform('x0', x0range[0], x0range[1]),
    'x1': hp.uniform('x1', x1range[0], x1range[1])
}

In [ ]:
def objective(params):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    x_vector=[params[i] for i in params]
    # Loss must be minimized
    loss = funct(x_vector)
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

In [ ]:
# Algorithm
tpe_algorithm = tpe.suggest
# Trials object to track progress
bayes_trials = Trials()
MAX_EVALS = 49

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials)
print(objective(best))

[1.0388105332515103, -1.0847119002187977]
24.912539651073132
[-9.306897028107548, 3.795015123459489]
124.0469734840585
[9.46453598969714, -7.99763114694251]
217.2409564891852
[0.7159373239645994, -1.0208328102955928]
25.639966130709535
[-6.775545860596875, -3.547896728847551]
30.67480557588163
[6.483686946398123, 8.971694161118421]
47.26496439348818
[-7.8078714922618015, 5.782287405764478]
118.15304809096013
[-3.0514326782226897, -6.909439655830378]
20.330688390804905
[7.845769605354821, 0.13548725212782742]
72.59723192519344
[6.373191964608097, 9.873761134771176]
77.22206955184286
[3.587527100697983, -1.2082127778868408]
4.7211672332023715
[8.666622698728585, -9.43697311097248]
224.3823300099965
[-6.789915787398599, 3.8196892101601243]
103.45335148098114
[-0.5762709124863985, -0.5333033101077245]
-4.414699314867706
[9.045193711885766, -1.5815873396785474]
87.96558757278352
[-9.231374415350928, 5.841572490730334]
156.32687815699424
[5.27924594554721, -0.6794053776176749]
23.85822338550

In [ ]:
bayes_trials

In [ ]:
# Run the experiments
X,min_found, best_points=Optimtm(plot_steps=False)
print("Minimum found:",min_found, " at ",best_points)

Minimum :  -25.72524324258832
x0    -1.313131
x1     0.101010
z    -25.725243
Name: 4350, dtype: float64
R2 insufficient -0.46
R2 insufficient -0.03


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.



R2 insufficient -0.24


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.



R2 insufficient 0.00


...

R2 insufficient 0.32
R2 insufficient -0.22
R2 insufficient 0.38


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (30

R2 insufficient 0.36
R2 insufficient -0.09
R2 insufficient 0.18


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption



R2 insufficient -0.11
Minimum found: -21.835512802509268  at  x0    -0.620433
x1     0.288876
z    -21.835513
Name: n0, dtype: float64
